In [1]:
## Read in data for Stocks
# Stock information pulled from Yahoo Finance as of April 22.  Looking back 50 days to see the growth rate of various stocks
# Maximize Rate of Return for Portfolio with various constraints related to Beta, Dividend, Total Costs,Balance of which 
# means Banking, Retail, Metals, Pharma, Health and Tech stocks to minimize risk and too much dependance on one sector

import csv
with open('Stocks.csv', newline='') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    

In [2]:
# Create dataframe
# define Variables

import pandas as pd
df = pd.read_csv('Stocks.csv', sep=",")
df[0:10]

,Symbol,Symbol link,Company Name,Last Price,Dividend,50-DMA,50-DMA Chg,50-DMA Chg %,Rate of Return,1yr Target Est,Beta,Sector,Metal/Bond,Health/Pharm,Tech,Bank,Retail
0,PLG,PLG,Platinum Group Metals Ltd.,5.11,0.0010,4.04,1.07,26.47%,0.2647,6.96,1.23,Metals,1,0,0,0,0
1,ABML,ABML,American Battery Metals Corporation,1.92,0.0010,1.66,0.26,15.59%,0.1559,-,0.87,Metals,1,0,0,0,0
2,SMTS,SMTS,Sierra Metals Inc.,3.37,0.0010,3.00,0.37,12.15%,0.1215,4,1.90,Metals,1,0,0,0,0
3,CLOV,CLOV,"Clover Health Investments, Corp.",8.77,0.0010,8.11,0.66,8.17%,0.0817,12.67,0.00,Health,0,1,0,0,0
4,WPM,WPM,Wheaton Precious Metals Corp.,42.77,0.0121,39.63,3.14,7.91%,0.0791,60.45,0.34,Metals,1,0,0,0,0
5,TSLA,TSLA,"Tesla, Inc.",729.40,0.0010,677.06,52.34,7.73%,0.0773,650.81,2.01,Tech,0,0,1,0,0
6,MSFT,MSFT,Microsoft Corporation,261.15,0.0087,242.84,18.31,7.54%,0.0754,275.89,0.79,Tech,0,0,1,0,0
7,AAPL,AAPL,Apple Inc.,134.32,0.0062,125.55,8.77,6.99%,0.0699,152.06,1.22,Tech,0,0,1,0,0
8,PFE,PFE,Pfizer Inc.,38.66,0.0404,36.25,2.41,6.64%,0.0664,40.69,0.65,Pharm,0,1,0,0,0
9,ARWR,ARWR,"Arrowhead Pharmaceuticals, Inc.",71.59,0.0010,67.21,4.38,6.51%,0.0651,91.17,1.27,Pharm,0,1,0,0,0


In [3]:
# Checking location finder that we will use later
df['Symbol'].iloc[0]

'PLG'

In [17]:
import numpy as np


In [122]:
# Create individual vectors of variables
# This will be helpful for constraint calculations

r=df['Rate of Return']
c=(df['Last Price'])
b=(df['Beta'])
d=(df['Dividend'])
MB=df['Metal/Bond']*df['Last Price'] # to ensure the portfolio is balance
HP=df['Health/Pharm']*df['Last Price']  # to ensure the portfolio is balance
T=df['Tech']*df['Last Price']  # to ensure the portfolio is balance
B=df['Bank']*df['Last Price']  # to ensure the portfolio is balance
R=df['Retail']*df['Last Price']  # to ensure the portfolio is balance
mm=df['Metal/Bond'] # to ensure the portfolio is balance
hh=df['Health/Pharm'] # to ensure the portfolio is balance
tt=df['Tech'] # to ensure the portfolio is balance
bb=df['Bank'] # to ensure the portfolio is balance
rr=df['Retail'] # to ensure the portfolio is balance




In [233]:
# Define variable for Optimization problem
r=np.array(r)
j=25  # number of Stocks to consider
r=r[0:j]
c=np.array(c)
c=c[0:j]
b=b[0:j]
d=d[0:j]
MB=MB[0:j]
HP=HP[0:j]
T=T[0:j]
B=B[0:j]
R=R[0:j]
mm=mm[0:j]
hh=hh[0:j]
tt=tt[0:j]
bb=bb[0:j]
rr=rr[0:j]
TC=100000 # Total cost of portfolio goal
D_min=0.005 # min dividend average threshold that the portfolio should achieve
D_max=0.010 # Max dividend average threshold that the portfolio should achieve
Beta=0.8 # Max Beta average threshold that the portfolio should achieve
BT=0.15


In [234]:
import numpy as np
from sympy import symbols, Matrix, Function, simplify, exp, hessian, solve, init_printing
init_printing()

sp_list=[]  # spend list to aggregate costs
beta_list=[]


In [264]:
import numpy as np
from scipy.optimize import minimize

# To maximize the rate of return we have to use negative of function and ensure the standard inequalities
# are less than for constriansts

def objective(x):  # the overall rate of return  (not a percentage number)
    return (np.dot(-x*c,-r))/(np.dot(-x,c))

def constraint1(x): # Total value of Stock portfolio to $xxx,xxxx
    
    return (np.dot(-c,-x)) - TC

def constraint2(x):  # keep Beta or b below some value to reduce risk or volatility
    return (np.dot(b,x*c))/(np.dot(x,c)) - Beta

def constraint3(x):   # ensure that the overall dividend value of porfolio is above some value (not a percentage number)
    return np.dot(x*c,d)/(np.dot(x,c)) - D_min

def constraint4(x):   # ensure that the portfolio is balance in terms of Metals/Bond;
    
    return (np.dot(x*c,mm))/(TC) - BT

def constraint5(x):   # ensure that the portfolio is balance in terms of Helath/Pharm;
    
    return (np.dot(x*c,hh))/(TC) - BT

def constraint6(x):   # ensure that the portfolio is balance in terms of Tech;
    
    return (np.dot(x*c,tt))/(TC) - BT

def constraint7(x):   # ensure that the portfolio is balance in terms of Bank;
    
    return (np.dot(x*c,bb))/(TC) - BT

def constraint8(x):   # ensure that the portfolio is balance in terms of Retail;
    
    return (np.dot(x*c,rr))/(TC) - BT

def constraint9(x):   # Max Dividend;
    
    return np.dot(-x*c,d)/(np.dot(x,c)) + D_max

def constraint10(x):  # Min Types of stocks ,eg, >2 metal stocks
    
    return (np.nansum((x*mm)/x)+ np.nansum((x*hh)/x) + np.nansum((x*tt)/x) + np.nansum((x*bb)/x) + np.nansum((x*rr)/x))-10

# initial guesses
n = 25
x0 = np.zeros(n)
x0[0] = 0.1
x0[1] = 0.1
x0[2]= 0.1
#x0[3]= 0.1

con1 = {'type': 'eq', 'fun': constraint1}
con2 = {'type': 'ineq', 'fun': constraint2}
con3 = {'type': 'ineq', 'fun': constraint3}
con4 = {'type': 'ineq', 'fun': constraint4}
con5 = {'type': 'ineq', 'fun': constraint5}
con6 = {'type': 'ineq', 'fun': constraint6}
con7 = {'type': 'ineq', 'fun': constraint7}
con8 = {'type': 'ineq', 'fun': constraint8}
con9 = {'type': 'ineq', 'fun': constraint9}
con10 = {'type': 'ineq', 'fun': constraint10}


cons = ([con1,con2,con3,con4,con5,con6,con7,con8,con9, con10])  
bnds = ((0, None), (0, None),(0,None),(0,None),(0, None), (0, None),(0,None),(0,None),(0, None), (0, None),
       (0, None), (0, None),(0,None),(0,None),(0, None), (0, None),(0,None),(0,None),(0, None), (0, None),
       (0, None), (0, None),(0,None),(0,None),(0, None))

solution = minimize(objective,x0,method='SLSQP',\
                    constraints=cons,bounds=bnds,options={'gtol': 1e-6, 'disp': True,"maxiter": 2000})
#bounds=bnds

C:\Users\mhodg\Anaconda3\lib\site-packages\ipykernel_launcher.py:74: OptimizeWarning: Unknown solver options: gtol


Optimization terminated successfully.    (Exit mode 0)
            Current function value: -0.12857905980278073
            Iterations: 6
            Function evaluations: 163
            Gradient evaluations: 6


In [265]:
#show initial objective
#print('Initial Objective: ' + str(objective(x0)))

x =solution.x
x=np.round(x,2)
o=abs(objective(x)) # take absolute of objective to reflect the true function objective

list.clear(sp_list)
# show final objective
print('Final Objective or Rate of Return is : ' + str(o))
print('Solution')
for i in range(j):
        print(float(x[i]), 'shares of '+ str(df['Symbol'].iloc[i]),'or ' + str(df['Company Name'].iloc[i]),
              ' and sector is ', str(df['Sector'].iloc[i] ), 'Total Cost =',round(x[i]*df['Last Price'].iloc[i],2))

        sp_list.append(x[i]*df['Last Price'].iloc[i] )

print('Total Spent is ', round(sum(sp_list),2))

Final Objective or Rate of Return is : 0.12857885510442038
Solution
6466.77 shares of PLG or Platinum Group Metals Ltd.  and sector is  Metals Total Cost = 33045.19
0.0 shares of ABML or American Battery Metals Corporation  and sector is  Metals Total Cost = 0.0
0.0 shares of SMTS or Sierra Metals Inc.  and sector is  Metals Total Cost = 0.0
923.43 shares of CLOV or Clover Health Investments, Corp.  and sector is  Health Total Cost = 8098.48
0.0 shares of WPM or Wheaton Precious Metals Corp.  and sector is  Metals Total Cost = 0.0
20.57 shares of TSLA or Tesla, Inc.  and sector is  Tech Total Cost = 15003.76
0.0 shares of MSFT or Microsoft Corporation  and sector is  Tech Total Cost = 0.0
0.0 shares of AAPL or Apple Inc.  and sector is  Tech Total Cost = 0.0
0.0 shares of PFE or Pfizer Inc.  and sector is  Pharm Total Cost = 0.0
96.41 shares of ARWR or Arrowhead Pharmaceuticals, Inc.  and sector is  Pharm Total Cost = 6901.99
72.81 shares of TGT or Target Corporation  and sector is  Re

In [266]:
from tabulate import tabulate
list.clear(beta_list)
# Print out constraint results
beta_list.append(np.dot(b,x))
                    
print('Beta Average for Portfolio is',round(sum(beta_list)/sum(x),2), 'verses goal of > than', Beta)
print('Dividend Average for Portfolio is',round(np.dot(x*c,d)/(np.dot(x,c)),6), 'verses goal of between',D_min, 'and',D_max)
print('Percent of Metal for Portfolio is',round(np.dot(x*c,mm)/(TC),4), 'verses goal of > than', BT)
print('Percent of Health/Pharma for Portfolio is',round(np.dot(x*c,hh)/(TC),4), 'verses goal of > than',BT)
print('Percent of Tech for Portfolio is',round(np.dot(x*c,tt)/(TC),4), 'verses goal of > than',BT)
print('Percent of Bank for Portfolio is',round(np.dot(x*c,bb)/(TC),4), 'verses goal of > than',BT)
print('Percent of Retail for Portfolio is',round((np.dot(x*c,rr))/(TC),4), 'verses goal of > than',BT)
z=0
for i in range(j):
    
    if x[i]>1: z=z+1
print('The number of Stocks greater than 1 is', z)

Beta Average for Portfolio is 1.09 verses goal of > than 0.8
Dividend Average for Portfolio is 0.005265 verses goal of between 0.005 and 0.01
Percent of Metal for Portfolio is 0.4 verses goal of > than 0.15
Percent of Health/Pharma for Portfolio is 0.15 verses goal of > than 0.15
Percent of Tech for Portfolio is 0.15 verses goal of > than 0.15
Percent of Bank for Portfolio is 0.15 verses goal of > than 0.15
Percent of Retail for Portfolio is 0.15 verses goal of > than 0.15
The number of Stocks greater than 1 is 7
